In [1]:
#import libraries
import sqlite3
from urllib.request import urlretrieve
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import os
import requests as req
import json
import time
import random
from riotwatcher import LolWatcher, ApiError

In [2]:
#Set our variables
api_key = 'RGAPI-a02bf935-a784-455f-80f3-bc0e85d01ba0'
watcher = LolWatcher(api_key)
my_region = 'na1'

In [3]:
#lets test the API 
me = watcher.summoner.by_name(my_region, 'Doublelift')
print(me)

{'id': 'Xxz7tMwFwICmvbNWTd5Ts-fdzn0YTMPSzBbYSusnPUMmy5c', 'accountId': '3o-WQejau7Mv8GPGFaNYU_Aro_YgN2NqLO1gi3nfsaKr-Q', 'puuid': 'nVctAvZgKZG9IwOPO-7p-T-VdoGP1VoyvAEmH9KGxiCN3lgb1zR36q-amdJGvKwhUoHeEafnj4mA8Q', 'name': 'Doublelift', 'profileIconId': 4568, 'revisionDate': 1599373000000, 'summonerLevel': 207}


In [51]:
#collect the data for all available seasons 
url_2020 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20200821.csv'
url_2019 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2019_LoL_esports_match_data_from_OraclesElixir_20201005.csv'
url_2018 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2018_LoL_esports_match_data_from_OraclesElixir_20201005.csv'
url_2017 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2017_LoL_esports_match_data_from_OraclesElixir_20201005.csv'
url_2016 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2016_LoL_esports_match_data_from_OraclesElixir_20201005.csv'
url_2015 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2015_LoL_esports_match_data_from_OraclesElixir_20201005.csv'
url_2014 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2014_LoL_esports_match_data_from_OraclesElixir_20201005.csv'

In [52]:
fullfilename_2020 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2020-LoL-matches.csv')
urlretrieve(url, fullfilename_2020)

fullfilename_2019 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2019-LoL-matches.csv')
urlretrieve(url, fullfilename_2019)

fullfilename_2018 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2018-LoL-matches.csv')
urlretrieve(url, fullfilename_2018)

fullfilename_2017 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2017-LoL-matches.csv')
urlretrieve(url, fullfilename_2017)

fullfilename_2016 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2016-LoL-matches.csv')
urlretrieve(url, fullfilename_2016)

fullfilename_2015 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2015-LoL-matches.csv')
urlretrieve(url, fullfilename_2015)

fullfilename_2014 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2014-LoL-matches.csv')
urlretrieve(url, fullfilename_2014)

('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external/2014-LoL-matches.csv',
 <http.client.HTTPMessage at 0x7ffb5014a6d0>)

In [53]:
#save to a dataframe
game_data_2020 = pd.read_csv(fullfilename_2020)
game_data_2019 = pd.read_csv(fullfilename_2019)
game_data_2018 = pd.read_csv(fullfilename_2018)
game_data_2017 = pd.read_csv(fullfilename_2017)
game_data_2016 = pd.read_csv(fullfilename_2016)
game_data_2015 = pd.read_csv(fullfilename_2015)
game_data_2014 = pd.read_csv(fullfilename_2014)

In [61]:
game_data_2020['season'] = '2020'
game_data_2019['season'] = '2019'
game_data_2018['season'] = '2018'
game_data_2017['season'] = '2017'
game_data_2016['season'] = '2016'
game_data_2015['season'] = '2015'
game_data_2014['season'] = '2014'

In [62]:
cols = list(game_data_2020)

In [63]:
print(cols)

['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare', 'wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm', 'totalgold', 'earnedgold', 'earned gpm', 'earnedgoldshare', 'gol

In [64]:
game_data_2019 = game_data_2019.rename(columns=dict(zip(game_data_2019.columns,cols)))
game_data_2018 = game_data_2018.rename(columns=dict(zip(game_data_2018.columns,cols)))
game_data_2017 = game_data_2017.rename(columns=dict(zip(game_data_2017.columns,cols)))
game_data_2016 = game_data_2016.rename(columns=dict(zip(game_data_2016.columns,cols)))
game_data_2015 = game_data_2015.rename(columns=dict(zip(game_data_2015.columns,cols)))
game_data_2014 = game_data_2014.rename(columns=dict(zip(game_data_2014.columns,cols)))

In [65]:
frames = [game_data_2020, game_data_2019, game_data_2018, game_data_2017, game_data_2016, game_data_2015, game_data_2014]

In [66]:
#concatenate the dataframes
game_data = pd.concat(frames)

In [67]:
#lets take a look at the data
game_data.info()
game_data.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438312 entries, 0 to 62615
Columns: 104 entries, gameid to season
dtypes: float64(67), int64(20), object(17)
memory usage: 351.1+ MB


gameid               object
datacompleteness     object
url                  object
league               object
year                  int64
                     ...   
opp_csat15          float64
golddiffat15        float64
xpdiffat15          float64
csdiffat15          float64
season               object
Length: 104, dtype: object

In [68]:
print(game_data.shape)

(438312, 104)


From here we can see there are 438,312 entries and 104 columns / features

In [69]:
#check for null values
game_data.isnull().values.any()

True

There are null entries, but rather than address them now we will address them a bit later

In [70]:
#lets isolate the number of league entries
game_data.league.unique()

array(['KeSPA', 'LPL', 'LFL', 'LEC', 'LCS.A', 'CBLOL', 'LCS', 'OPL',
       'VCS', 'TCL', 'TRA', 'BRCC', 'LCK', 'CK', 'LJL', 'LLA', 'LCL',
       'PCS', 'EM', 'LDL', 'OCS', 'Riot', 'MSC'], dtype=object)

We want to eventually isolate the LCS. We note that the LCS.A is the Academy/minor league, so we ignore that entry later on



In [71]:
#make sure year is datetime object
game_data['year'] = pd.to_datetime(game_data['year'])
game_data.dtypes

gameid                      object
datacompleteness            object
url                         object
league                      object
year                datetime64[ns]
                         ...      
opp_csat15                 float64
golddiffat15               float64
xpdiffat15                 float64
csdiffat15                 float64
season                      object
Length: 104, dtype: object

In [72]:
#do the same for playerid, game, and playoffs: make sure they are categories
game_data['playerid'] = game_data['playerid'].astype('category')
game_data['game'] = game_data['game'].astype('category')
game_data['playoffs'] = game_data['playoffs'].astype('category')

In [73]:
#lets take a look at the first ten entries of our dataframe
game_data.head(10)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4888.0,7368.0,131.0,4723.0,7202.0,118.0,165.0,166.0,13.0,2020
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4385.0,4817.0,91.0,4784.0,4667.0,98.0,-399.0,150.0,-7.0,2020
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4809.0,6275.0,129.0,5218.0,8112.0,140.0,-409.0,-1837.0,-11.0,2020
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4915.0,4959.0,136.0,4864.0,5360.0,135.0,51.0,-401.0,1.0,2020
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,2956.0,3800.0,20.0,3189.0,3543.0,28.0,-233.0,257.0,-8.0,2020
5,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4723.0,7202.0,118.0,4888.0,7368.0,131.0,-165.0,-166.0,-13.0,2020
6,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4784.0,4667.0,98.0,4385.0,4817.0,91.0,399.0,-150.0,7.0,2020
7,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,5218.0,8112.0,140.0,4809.0,6275.0,129.0,409.0,1837.0,11.0,2020
8,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4864.0,5360.0,135.0,4915.0,4959.0,136.0,-51.0,401.0,-1.0,2020
9,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,3189.0,3543.0,28.0,2956.0,3800.0,20.0,233.0,-257.0,8.0,2020


There are more than two entries per game, despite there only being 2 teams. This is because our dataframe currently aggregates match statistics for individual players -- we care only about the team. In this notebook we will isolate the team entries.

In [74]:
game_data[game_data.gameid == 'ESPORTSTMNT03/1241318'].T

,0,1,2,3,4,5,6,7,8,9,...,2,3,4,5,6,7,8,9,10,11
gameid,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,...,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318
datacompleteness,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,...,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete
url,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...
league,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,...,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA
year,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,...,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
opp_csat15,118,98,140,135,28,131,91,129,136,20,...,140,135,28,131,91,129,136,20,519,507
golddiffat15,165,-399,-409,51,-233,-165,399,409,-51,233,...,-409,51,-233,-165,399,409,-51,233,-825,825
xpdiffat15,166,150,-1837,-401,257,-166,-150,1837,401,-257,...,-1837,-401,257,-166,-150,1837,401,-257,-1665,1665
csdiffat15,13,-7,-11,1,-8,-13,7,11,-1,8,...,-11,1,-8,-13,7,11,-1,8,-12,12


In fact, there are 12 entries for the first game listed: 5 team members, one team entry (and 2 teams. Let's explore the datatypes of the columns

In [75]:
game_data.select_dtypes(include='object')

,gameid,datacompleteness,url,league,split,date,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,season
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,top,Summit,SANDBOX Gaming,Rumble,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2020
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,jng,OnFleek,SANDBOX Gaming,Elise,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2020
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,mid,FATE,SANDBOX Gaming,Qiyana,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2020
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,bot,Leo,SANDBOX Gaming,Miss Fortune,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2020
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,sup,GorillA,SANDBOX Gaming,Nautilus,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62611,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Red,mid,Claire,Chiefs Esports Club,Akali,Thresh,Caitlyn,Karma,Lulu,Morgana,2014
62612,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Red,bot,Katsurii,Chiefs Esports Club,Ashe,Thresh,Caitlyn,Karma,Lulu,Morgana,2014
62613,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Red,sup,Dragku,Chiefs Esports Club,Sett,Thresh,Caitlyn,Karma,Lulu,Morgana,2014
62614,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Blue,team,NaN,Pentanet.GG,NaN,Nidalee,Olaf,Yuumi,Gangplank,Jayce,2014


In [76]:
game_data[game_data.gameid == 'ESPORTSTMNT03/1241318'].T.head(26)

,0,1,2,3,4,5,6,7,8,9,...,2,3,4,5,6,7,8,9,10,11
gameid,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,...,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318
datacompleteness,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,...,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete
url,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...
league,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,...,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA
year,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,...,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020
split,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
playoffs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
date,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,...,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26
game,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
patch,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,...,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24


Lets now isolate our league of interest

In [77]:
#isolate LCS
array = ['LCS']
LCS_data = game_data.loc[game_data['league'].isin(array)]
print(LCS_data.head())
LCS_data.info()

                    gameid datacompleteness  \
864  ESPORTSTMNT02/1270555         complete   
865  ESPORTSTMNT02/1270555         complete   
866  ESPORTSTMNT02/1270555         complete   
867  ESPORTSTMNT02/1270555         complete   
868  ESPORTSTMNT02/1270555         complete   

                                                   url league  \
864  https://matchhistory.na.leagueoflegends.com/en...    LCS   
865  https://matchhistory.na.leagueoflegends.com/en...    LCS   
866  https://matchhistory.na.leagueoflegends.com/en...    LCS   
867  https://matchhistory.na.leagueoflegends.com/en...    LCS   
868  https://matchhistory.na.leagueoflegends.com/en...    LCS   

                             year   split playoffs                 date game  \
864 1970-01-01 00:00:00.000002020  Spring        0  2020-01-25 21:37:53    1   
865 1970-01-01 00:00:00.000002020  Spring        0  2020-01-25 21:37:53    1   
866 1970-01-01 00:00:00.000002020  Spring        0  2020-01-25 21:37:53    1   
867 19

In [78]:
#check for null columns
null_columns = LCS_data.columns[LCS_data.isnull().any()]
LCS_data[null_columns].isnull().sum().sort_values(ascending=False)

dragons (type unknown)    17563
elementaldrakes           16450
opp_elementaldrakes       16450
infernals                 16450
mountains                 16450
clouds                    16450
oceans                    16450
elders                    16380
firstherald               16380
dragons                   16380
opp_dragons               16380
gspd                      16380
opp_elders                16380
heralds                   16380
opp_heralds               16380
firstbaron                16380
barons                    16380
opp_barons                16380
firsttower                16380
towers                    16380
opp_towers                16380
firstmidtower             16380
firsttothreetowers        16380
firstdragon               16380
firstbloodassist           3276
firstbloodkill             3276
champion                   3276
total cs                   3276
firstbloodvictim           3276
damageshare                3276
earnedgoldshare            3276
player  

We note that the features with 16380+ null entries are team statistics -- meaning there are entries for each team but not for the players, as teams get credited with those game accomplishments. This means when we isolate the team entries these should no longer show up as null.

In [79]:
#find the rows with null game ids
LCS_data[LCS_data["gameid"].isnull()]

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season


We find that there are no missing game IDs in our LCS dataframe, which is good (and verified above as well)

In [80]:
LCS_data[LCS_data['player'].isnull()].head(10000)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2020
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2020
886,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,23196.0,26241.0,486.0,24778.0,29394.0,522.0,-1582.0,-3153.0,-36.0,2020
887,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,24778.0,29394.0,522.0,23196.0,26241.0,486.0,1582.0,3153.0,36.0,2020
898,ESPORTSTMNT02/1270592,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 23:39:30,1,10.01,...,24058.0,26322.0,461.0,22600.0,26940.0,461.0,1458.0,-618.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62555,ESPORTSTMNT03/1441723,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 20:35:30,2,10.16,...,25952.0,28599.0,453.0,25950.0,28193.0,475.0,2.0,406.0,-22.0,2014
62566,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,24457.0,29149.0,493.0,23168.0,28698.0,490.0,1289.0,451.0,3.0,2014
62567,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,23168.0,28698.0,490.0,24457.0,29149.0,493.0,-1289.0,-451.0,-3.0,2014
62578,ESPORTSTMNT03/1441735,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 23:33:01,4,10.16,...,24352.0,28436.0,462.0,23542.0,28120.0,460.0,810.0,316.0,2.0,2014


These entries likely correspond to entries for the "team" position, for which there are null stats on the individual player features... so lets filter further to confirm

In [81]:
#get numerical statistics
LCS_data.describe().T

,count,mean,std,min,25%,50%,75%,max
patch,19656.0,10.085598,0.050797,10.01,10.04,10.06,10.14,10.16
gamelength,19656.0,2047.807692,391.482078,1337.00,1779.00,1964.50,2286.00,3680.00
result,19656.0,0.500000,0.500013,0.00,0.00,0.50,1.00,1.00
kills,19656.0,3.725783,4.640875,0.00,1.00,2.00,5.00,29.00
deaths,19656.0,3.729345,4.447987,0.00,1.00,2.00,4.00,29.00
...,...,...,...,...,...,...,...,...
opp_xpat15,19656.0,9669.880342,8769.865040,2681.00,4862.25,6308.50,7710.50,33530.00
opp_csat15,19656.0,169.306268,157.739522,0.00,89.00,126.00,149.00,594.00
golddiffat15,19656.0,0.000000,1301.485240,-7782.00,-555.25,0.00,555.25,7782.00
xpdiffat15,19656.0,0.000000,988.542284,-5909.00,-506.00,0.00,506.00,5909.00


In [82]:
#lets look for missing entries:
missing = pd.concat([LCS_data.isnull().sum(), 100 * LCS_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)
missing.head()

,count,%
gameid,0,0.0
datacompleteness,0,0.0
url,0,0.0
league,0,0.0
year,0,0.0


In [84]:
#lets find the number of unique games ### come back to this
print(LCS_data['gameid'].nunique())

234


234 unique games

In [85]:
#isolate team performances
team = LCS_data.position == 'team'
LCS_data_by_team = LCS_data[team]

In [86]:
LCS_data_by_team['team'].value_counts()

FlyQuest                455
Evil Geniuses           399
Team SoloMid            364
Cloud9                  350
100 Thieves             343
Golden Guardians        301
Dignitas                294
Counter Logic Gaming    259
Immortals               259
Team Liquid             252
Name: team, dtype: int64

In [87]:
#lets store this in a dictionary for later use
game_counts = LCS_data_by_team['team'].value_counts().to_dict()

In [88]:
print(game_counts)

{'FlyQuest': 455, 'Evil Geniuses': 399, 'Team SoloMid': 364, 'Cloud9': 350, '100 Thieves': 343, 'Golden Guardians': 301, 'Dignitas': 294, 'Counter Logic Gaming': 259, 'Immortals': 259, 'Team Liquid': 252}


In [89]:
#run the code above to confirm that the null entries were from team stats
LCS_data_by_team[LCS_data_by_team['player'].isnull()].head(10000)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2020
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2020
886,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,23196.0,26241.0,486.0,24778.0,29394.0,522.0,-1582.0,-3153.0,-36.0,2020
887,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,24778.0,29394.0,522.0,23196.0,26241.0,486.0,1582.0,3153.0,36.0,2020
898,ESPORTSTMNT02/1270592,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 23:39:30,1,10.01,...,24058.0,26322.0,461.0,22600.0,26940.0,461.0,1458.0,-618.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62555,ESPORTSTMNT03/1441723,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 20:35:30,2,10.16,...,25952.0,28599.0,453.0,25950.0,28193.0,475.0,2.0,406.0,-22.0,2014
62566,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,24457.0,29149.0,493.0,23168.0,28698.0,490.0,1289.0,451.0,3.0,2014
62567,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,23168.0,28698.0,490.0,24457.0,29149.0,493.0,-1289.0,-451.0,-3.0,2014
62578,ESPORTSTMNT03/1441735,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 23:33:01,4,10.16,...,24352.0,28436.0,462.0,23542.0,28120.0,460.0,810.0,316.0,2.0,2014


In [91]:
LCS_data_by_team.head(10000)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2020
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2020
886,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,23196.0,26241.0,486.0,24778.0,29394.0,522.0,-1582.0,-3153.0,-36.0,2020
887,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,24778.0,29394.0,522.0,23196.0,26241.0,486.0,1582.0,3153.0,36.0,2020
898,ESPORTSTMNT02/1270592,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 23:39:30,1,10.01,...,24058.0,26322.0,461.0,22600.0,26940.0,461.0,1458.0,-618.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62555,ESPORTSTMNT03/1441723,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 20:35:30,2,10.16,...,25952.0,28599.0,453.0,25950.0,28193.0,475.0,2.0,406.0,-22.0,2014
62566,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,24457.0,29149.0,493.0,23168.0,28698.0,490.0,1289.0,451.0,3.0,2014
62567,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,23168.0,28698.0,490.0,24457.0,29149.0,493.0,-1289.0,-451.0,-3.0,2014
62578,ESPORTSTMNT03/1441735,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 23:33:01,4,10.16,...,24352.0,28436.0,462.0,23542.0,28120.0,460.0,810.0,316.0,2.0,2014


In [93]:
#isolate stats for the first game of the first match in the dataframe
first_game = LCS_data_by_team.gameid == 'ESPORTSTMNT02/1270555'
LCS_data_first_game = LCS_data_by_team[first_game]
LCS_data_first_game.head(500)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2020
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2020
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2019
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2019
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2018
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2018
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2017
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2017
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2016
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2016


In [94]:
#select all numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
newdf = LCS_data_first_game.select_dtypes(include=numerics)
all_stats = newdf.columns

In [95]:
#group totals by teamname
team_totals = LCS_data_first_game.groupby(by='team')[all_stats].sum().round(2)
print(team_totals.head(12))
print(team_totals.info())


             patch  gamelength  result  kills  deaths  assists  teamkills  \
team                                                                        
Cloud9       70.07       10318       7     63      21      126         63   
Team Liquid  70.07       10318       0     21      63       42         21   

             teamdeaths  doublekills  triplekills  ...  csdiffat10  goldat15  \
team                                               ...                         
Cloud9               21          7.0          0.0  ...       -49.0  160643.0   
Team Liquid          63          0.0          0.0  ...        49.0  160965.0   

               xpat15  csat15  opp_goldat15  opp_xpat15  opp_csat15  \
team                                                                  
Cloud9       200501.0  3395.0      160965.0    192738.0      3367.0   
Team Liquid  192738.0  3367.0      160643.0    200501.0      3395.0   

             golddiffat15  xpdiffat15  csdiffat15  
team                             

In [96]:
team_totals = LCS_data_by_team.groupby(by='team')[all_stats].sum().round(2)
print(team_totals.head(12))
print(team_totals.info())

                        patch  gamelength  result  kills  deaths  assists  \
team                                                                        
100 Thieves           3458.42      691425     140   3738    4375     8491   
Cloud9                3529.54      647248     280   5425    3010    12544   
Counter Logic Gaming  2611.91      536522      56   2520    3724     6321   
Dignitas              2966.11      606053     105   2730    3248     6517   
Evil Geniuses         4025.28      801906     203   4382    4921    10241   
FlyQuest              4589.34      930181     266   5810    4774    14399   
Golden Guardians      3035.90      628159     147   2912    3577     6986   
Immortals             2611.28      560413      84   2086    3234     5005   
Team Liquid           2540.86      533869     154   2695    2065     6573   
Team SoloMid          3671.78      772842     203   4319    3724    10290   

                      teamkills  teamdeaths  doublekills  triplekills  ... 

In [97]:
team_totals['gamesplayed']= pd.Series(game_counts)

In [98]:
team_totals['gamesplayed'].head(20)

team
100 Thieves             343
Cloud9                  350
Counter Logic Gaming    259
Dignitas                294
Evil Geniuses           399
FlyQuest                455
Golden Guardians        301
Immortals               259
Team Liquid             252
Team SoloMid            364
Name: gamesplayed, dtype: int64

In [99]:
team_totals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 100 Thieves to Team SoloMid
Data columns (total 84 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   patch                    10 non-null     float64
 1   gamelength               10 non-null     int64  
 2   result                   10 non-null     int64  
 3   kills                    10 non-null     int64  
 4   deaths                   10 non-null     int64  
 5   assists                  10 non-null     int64  
 6   teamkills                10 non-null     int64  
 7   teamdeaths               10 non-null     int64  
 8   doublekills              10 non-null     float64
 9   triplekills              10 non-null     float64
 10  quadrakills              10 non-null     float64
 11  pentakills               10 non-null     float64
 12  firstblood               10 non-null     float64
 13  firstbloodkill           10 non-null     float64
 14  firstbloodass

Let's move the cells so that the game totals are first, and the results second, so we can compute wins and losses. 

In [100]:
# get a list of columns
cols = list(team_totals)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('gamesplayed')))
cols

['gamesplayed',
 'patch',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'firstdragon',
 'dragons',
 'opp_dragons',
 'elementaldrakes',
 'opp_elementaldrakes',
 'infernals',
 'mountains',
 'clouds',
 'oceans',
 'dragons (type unknown)',
 'elders',
 'opp_elders',
 'firstherald',
 'heralds',
 'opp_heralds',
 'firstbaron',
 'barons',
 'opp_barons',
 'firsttower',
 'towers',
 'opp_towers',
 'firstmidtower',
 'firsttothreetowers',
 'inhibitors',
 'opp_inhibitors',
 'damagetochampions',
 'dpm',
 'damageshare',
 'wardsplaced',
 'wpm',
 'wardskilled',
 'wcpm',
 'controlwardsbought',
 'visionscore',
 'vspm',
 'totalgold',
 'earnedgold',
 'earned gpm',
 'earnedgoldshare',
 'goldspent',
 'gspd',
 'total cs',
 'minionkills',
 'monsterkills',
 'monsterkillsownjungle',
 'monsterkillsenemyjungle',
 'c

In [101]:
# use ix to reorder
team_totals = team_totals.loc[:, cols]
team_totals

,gamesplayed,patch,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,3458.42,691425,140,3738,4375,8491,3738,4375,413.0,...,-693.0,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0
Cloud9,350,3529.54,647248,280,5425,3010,12544,5425,3010,637.0,...,5761.0,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0
Counter Logic Gaming,259,2611.91,536522,56,2520,3724,6321,2520,3724,189.0,...,-973.0,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0
Dignitas,294,2966.11,606053,105,2730,3248,6517,2730,3248,287.0,...,-2247.0,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0
Evil Geniuses,399,4025.28,801906,203,4382,4921,10241,4382,4921,406.0,...,-3269.0,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0
FlyQuest,455,4589.34,930181,266,5810,4774,14399,5810,4774,623.0,...,-2170.0,11053343.0,13185690.0,230419.0,10982909.0,13085723.0,230202.0,70434.0,99967.0,217.0
Golden Guardians,301,3035.90,628159,147,2912,3577,6986,2912,3577,301.0,...,3304.0,7317716.0,8867243.0,158515.0,7181076.0,8773464.0,153748.0,136640.0,93779.0,4767.0
Immortals,259,2611.28,560413,84,2086,3234,5005,2086,3234,154.0,...,-2583.0,6066032.0,7349671.0,129857.0,6306727.0,7535388.0,134295.0,-240695.0,-185717.0,-4438.0
Team Liquid,252,2540.86,533869,154,2695,2065,6573,2695,2065,280.0,...,2107.0,6103783.0,7358715.0,130389.0,6016010.0,7235914.0,127113.0,87773.0,122801.0,3276.0


In [102]:
# get a list of columns
cols = list(team_totals)
# move the column to head of list using index, pop and insert
cols.insert(1, cols.pop(cols.index('result')))
team_totals = team_totals.loc[:, cols]
team_totals

,gamesplayed,result,patch,gamelength,kills,deaths,assists,teamkills,teamdeaths,doublekills,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,140,3458.42,691425,3738,4375,8491,3738,4375,413.0,...,-693.0,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0
Cloud9,350,280,3529.54,647248,5425,3010,12544,5425,3010,637.0,...,5761.0,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0
Counter Logic Gaming,259,56,2611.91,536522,2520,3724,6321,2520,3724,189.0,...,-973.0,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0
Dignitas,294,105,2966.11,606053,2730,3248,6517,2730,3248,287.0,...,-2247.0,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0
Evil Geniuses,399,203,4025.28,801906,4382,4921,10241,4382,4921,406.0,...,-3269.0,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0
FlyQuest,455,266,4589.34,930181,5810,4774,14399,5810,4774,623.0,...,-2170.0,11053343.0,13185690.0,230419.0,10982909.0,13085723.0,230202.0,70434.0,99967.0,217.0
Golden Guardians,301,147,3035.90,628159,2912,3577,6986,2912,3577,301.0,...,3304.0,7317716.0,8867243.0,158515.0,7181076.0,8773464.0,153748.0,136640.0,93779.0,4767.0
Immortals,259,84,2611.28,560413,2086,3234,5005,2086,3234,154.0,...,-2583.0,6066032.0,7349671.0,129857.0,6306727.0,7535388.0,134295.0,-240695.0,-185717.0,-4438.0
Team Liquid,252,154,2540.86,533869,2695,2065,6573,2695,2065,280.0,...,2107.0,6103783.0,7358715.0,130389.0,6016010.0,7235914.0,127113.0,87773.0,122801.0,3276.0


In [103]:
team_totals.insert(2, 'Losses', 0)

In [104]:
team_totals.head()

,gamesplayed,result,Losses,patch,gamelength,kills,deaths,assists,teamkills,teamdeaths,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,140,0,3458.42,691425,3738,4375,8491,3738,4375,...,-693.0,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0
Cloud9,350,280,0,3529.54,647248,5425,3010,12544,5425,3010,...,5761.0,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0
Counter Logic Gaming,259,56,0,2611.91,536522,2520,3724,6321,2520,3724,...,-973.0,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0
Dignitas,294,105,0,2966.11,606053,2730,3248,6517,2730,3248,...,-2247.0,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0
Evil Geniuses,399,203,0,4025.28,801906,4382,4921,10241,4382,4921,...,-3269.0,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0


In [105]:
mapping = {team_totals.columns[1]: 'Wins'}
team_totals = team_totals.rename(columns=mapping)
team_totals.head()

,gamesplayed,Wins,Losses,patch,gamelength,kills,deaths,assists,teamkills,teamdeaths,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,140,0,3458.42,691425,3738,4375,8491,3738,4375,...,-693.0,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0
Cloud9,350,280,0,3529.54,647248,5425,3010,12544,5425,3010,...,5761.0,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0
Counter Logic Gaming,259,56,0,2611.91,536522,2520,3724,6321,2520,3724,...,-973.0,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0
Dignitas,294,105,0,2966.11,606053,2730,3248,6517,2730,3248,...,-2247.0,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0
Evil Geniuses,399,203,0,4025.28,801906,4382,4921,10241,4382,4921,...,-3269.0,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0


In [42]:
team_totals.insert(3, 'Win_Percentage', 0)
team_totals.head()

,gamesplayed,Wins,Losses,Win_Percentage,patch,gamelength,kills,deaths,assists,teamkills,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,49,20,0,0,494.06,98775,534,625,1213,534,...,-99.0,1159872.0,1402835.0,24271.0,1196839.0,1427357.0,24539.0,-36967.0,-24522.0,-268.0
Cloud9,50,40,0,0,504.22,92464,775,430,1792,775,...,823.0,1280844.0,1491829.0,25510.0,1162712.0,1409222.0,24201.0,118132.0,82607.0,1309.0
Counter Logic Gaming,37,8,0,0,373.13,76646,360,532,903,360,...,-139.0,869714.0,1069385.0,18978.0,926688.0,1108046.0,19352.0,-56974.0,-38661.0,-374.0
Dignitas,42,15,0,0,423.73,86579,390,464,931,390,...,-321.0,1007052.0,1219211.0,21575.0,1028798.0,1237821.0,22083.0,-21746.0,-18610.0,-508.0
Evil Geniuses,57,29,0,0,575.04,114558,626,703,1463,626,...,-467.0,1362152.0,1639762.0,28258.0,1388464.0,1657467.0,28956.0,-26312.0,-17705.0,-698.0


In [106]:
team_totals['Losses'] = team_totals['gamesplayed'] - team_totals['Wins'] 
team_totals.head()

,gamesplayed,Wins,Losses,patch,gamelength,kills,deaths,assists,teamkills,teamdeaths,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,140,203,3458.42,691425,3738,4375,8491,3738,4375,...,-693.0,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0
Cloud9,350,280,70,3529.54,647248,5425,3010,12544,5425,3010,...,5761.0,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0
Counter Logic Gaming,259,56,203,2611.91,536522,2520,3724,6321,2520,3724,...,-973.0,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0
Dignitas,294,105,189,2966.11,606053,2730,3248,6517,2730,3248,...,-2247.0,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0
Evil Geniuses,399,203,196,4025.28,801906,4382,4921,10241,4382,4921,...,-3269.0,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0


In [107]:
team_totals['Win_Percentage'] = team_totals['Wins'] / team_totals['gamesplayed']

,gamesplayed,Wins,Losses,patch,gamelength,kills,deaths,assists,teamkills,teamdeaths,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,Win_Percentage
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,140,203,3458.42,691425,3738,4375,8491,3738,4375,...,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0,0.408163
Cloud9,350,280,70,3529.54,647248,5425,3010,12544,5425,3010,...,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0,0.800000
Counter Logic Gaming,259,56,203,2611.91,536522,2520,3724,6321,2520,3724,...,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0,0.216216
Dignitas,294,105,189,2966.11,606053,2730,3248,6517,2730,3248,...,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0,0.357143
Evil Geniuses,399,203,196,4025.28,801906,4382,4921,10241,4382,4921,...,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0,0.508772


In [108]:
team_totals.head(1000)

,gamesplayed,Wins,Losses,patch,gamelength,kills,deaths,assists,teamkills,teamdeaths,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,Win_Percentage
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,140,203,3458.42,691425,3738,4375,8491,3738,4375,...,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0,0.408163
Cloud9,350,280,70,3529.54,647248,5425,3010,12544,5425,3010,...,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0,0.800000
Counter Logic Gaming,259,56,203,2611.91,536522,2520,3724,6321,2520,3724,...,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0,0.216216
Dignitas,294,105,189,2966.11,606053,2730,3248,6517,2730,3248,...,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0,0.357143
Evil Geniuses,399,203,196,4025.28,801906,4382,4921,10241,4382,4921,...,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0,0.508772
FlyQuest,455,266,189,4589.34,930181,5810,4774,14399,5810,4774,...,11053343.0,13185690.0,230419.0,10982909.0,13085723.0,230202.0,70434.0,99967.0,217.0,0.584615
Golden Guardians,301,147,154,3035.90,628159,2912,3577,6986,2912,3577,...,7317716.0,8867243.0,158515.0,7181076.0,8773464.0,153748.0,136640.0,93779.0,4767.0,0.488372
Immortals,259,84,175,2611.28,560413,2086,3234,5005,2086,3234,...,6066032.0,7349671.0,129857.0,6306727.0,7535388.0,134295.0,-240695.0,-185717.0,-4438.0,0.324324
Team Liquid,252,154,98,2540.86,533869,2695,2065,6573,2695,2065,...,6103783.0,7358715.0,130389.0,6016010.0,7235914.0,127113.0,87773.0,122801.0,3276.0,0.611111


Things left to do:
3) Aggregate the player data
4) Export the CSV

In [111]:
cols = list(team_totals)
cols.insert(1, cols.pop(cols.index('Win_Percentage')))
team_totals = team_totals.loc[:, cols]
team_totals

,gamesplayed,Win_Percentage,Wins,Losses,patch,gamelength,kills,deaths,assists,teamkills,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
team,,,,,,,,,,,,,,,,,,,,,
100 Thieves,343,0.408163,140,203,3458.42,691425,3738,4375,8491,3738,...,-693.0,8119104.0,9819845.0,169897.0,8377873.0,9991499.0,171773.0,-258769.0,-171654.0,-1876.0
Cloud9,350,0.800000,280,70,3529.54,647248,5425,3010,12544,5425,...,5761.0,8965908.0,10442803.0,178570.0,8138984.0,9864554.0,169407.0,826924.0,578249.0,9163.0
Counter Logic Gaming,259,0.216216,56,203,2611.91,536522,2520,3724,6321,2520,...,-973.0,6087998.0,7485695.0,132846.0,6486816.0,7756322.0,135464.0,-398818.0,-270627.0,-2618.0
Dignitas,294,0.357143,105,189,2966.11,606053,2730,3248,6517,2730,...,-2247.0,7049364.0,8534477.0,151025.0,7201586.0,8664747.0,154581.0,-152222.0,-130270.0,-3556.0
Evil Geniuses,399,0.508772,203,196,4025.28,801906,4382,4921,10241,4382,...,-3269.0,9535064.0,11478334.0,197806.0,9719248.0,11602269.0,202692.0,-184184.0,-123935.0,-4886.0
FlyQuest,455,0.584615,266,189,4589.34,930181,5810,4774,14399,5810,...,-2170.0,11053343.0,13185690.0,230419.0,10982909.0,13085723.0,230202.0,70434.0,99967.0,217.0
Golden Guardians,301,0.488372,147,154,3035.90,628159,2912,3577,6986,2912,...,3304.0,7317716.0,8867243.0,158515.0,7181076.0,8773464.0,153748.0,136640.0,93779.0,4767.0
Immortals,259,0.324324,84,175,2611.28,560413,2086,3234,5005,2086,...,-2583.0,6066032.0,7349671.0,129857.0,6306727.0,7535388.0,134295.0,-240695.0,-185717.0,-4438.0
Team Liquid,252,0.611111,154,98,2540.86,533869,2695,2065,6573,2695,...,2107.0,6103783.0,7358715.0,130389.0,6016010.0,7235914.0,127113.0,87773.0,122801.0,3276.0


In [115]:
#eliminate any columns that have 0s
team_totals = team_totals.loc[:, (team_totals != 0).any(axis=0)]

In [116]:
#export to CSV file to be used for EDA
team_totals.to_csv(r'/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/processed/Wrangled_LoL_Data.csv')

This dataframe is ready for EDA